In [1]:
import os
import matplotlib.pyplot as plt
import time, os, json
import pandas as pd
from scipy import stats
import scipy
from tqdm import tqdm
import seaborn as sns
import pdb
import optax
from functools import partial
import math
import csv
import time
import numpy
from pdb import set_trace
from dataclasses import dataclass

import jax
from jax import random 
from jax.config import config
import jax.numpy as np
from jax import vmap
from jax.lib import xla_bridge

print("jax backend {}".format(xla_bridge.get_backend().platform))

from SSN_classes import SSN2DTopoV1_ONOFF_local, SSN2DTopoV1
from util import take_log, load_param_from_csv
from analysis import param_ratios_two_layer
from visualization import (
    plot_losses,
    plot_losses_two_stage,
    plot_results_two_layers,
    plot_sigmoid_outputs,
    plot_training_accs,
)
from training import *
from parameters import *

C:\Users\jozsa\AppData\Local\Temp\ipykernel_1556\109909311.py:21: DeprecationWarning: Accessing jax.config via the jax.config submodule is deprecated.
  from jax.config import config


jax backend cpu


In [2]:
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".XX"
os.environ["XLA_PYTHON_CLIENT_ALLOCATOR"] = "platform"

In [3]:
# Calculate find A and save
# BW_Grating(JiaGrating) is called 81 times and spatial_frequency is always 0.05235987755982988
ssn_mid = SSN2DTopoV1_ONOFF_local(
    ssn_pars=ssn_pars, # got c_E and c_I as new parameters
    grid_pars=grid_pars,
    conn_pars=conn_pars_m,
    filter_pars=filter_pars,
    J_2x2 = ssn_layer_pars.J_2x2_m,
    gE = ssn_layer_pars.gE_m, 
    gI = ssn_layer_pars.gE_s
)

ssn_pars.A = ssn_mid.A
if ssn_pars.phases == 4:
    ssn_pars.A2 = ssn_mid.A2


In [4]:
# Load orientation map, MJ
# ssn_ori_map = np.load(os.path.join(os.getcwd(), "ssn_map_uniform_good.npy"))

####################### TRAINING PARAMETERS #############################
epochs_to_save = np.insert(
    (np.unique(np.linspace(1, training_pars.epochs, training_pars.num_epochs_to_save).astype(int))), 0, 0
)

constant_ssn_pars = dict(
    ssn_pars=ssn_pars,
    grid_pars=grid_pars,
    conn_pars_m=conn_pars_m,
    conn_pars_s=conn_pars_s,
    gE=ssn_layer_pars.gE,
    gI=ssn_layer_pars.gI,
    filter_pars=filter_pars,
    conv_pars=conv_pars,# this got a lot of new stuff in it that might be a problem
    loss_pars=loss_pars,
    noise_type="poisson",
)


##################### SAVE RESULTS ###############################

# Name of results csv
home_dir = os.getcwd()

# Specify folder to save results
results_dir = os.path.join(
    home_dir,
    "results",
    "16-10",
    "phases_"
    + str(ssn_pars.phases)
    + "k_"
    + str(filter_pars.k)
    + "sigma_g"
    + str(filter_pars.sigma_g)
    + "gE_"
    + str(ssn_layer_pars.gE_m),
)

if os.path.exists(results_dir) == False:
    os.makedirs(results_dir)

# Specify results filename
run_dir = os.path.join(
    results_dir,
    "set_"
    + str("C") # init_set_m = "C" but I removed it for now
    + "_sig_noise_"
    + str(training_pars.sig_noise)
    + "_batch"
    + str(training_pars.batch_size)
    + "_lamw"
    + str(loss_pars.lambda_w),
)

# results_filename = None
results_filename = os.path.join(run_dir + "_results.csv")

In [5]:
########### TRAINING ############
(
    [ssn_layer_pars, readout_pars],
    val_loss_per_epoch,
    training_losses,
    training_accs,
    train_sig_inputs,
    train_sig_outputs,
    val_sig_inputs,
    val_sig_outputs,
    epoch_c,
    save_w_sigs,
) = new_two_stage_training(
    ssn_layer_pars,
    sig_pars,
    training_pars,
    constant_ssn_pars,
    stimuli_pars,
    results_filename=results_filename,
    results_dir=run_dir,
    extra_stop=2,
    # ssn_ori_map=ssn_ori_map,
)

print("new_pars ", ssn_layer_pars, readout_pars)

# Save training and validation losses
np.save(os.path.join(run_dir + "_training_losses.npy"), training_losses)
np.save(os.path.join(run_dir + "_validation_losses.npy"), val_loss_per_epoch)

# Plot losses
losses_dir = os.path.join(run_dir + "_losses")
plot_losses_two_stage(
    training_losses, val_loss_per_epoch, epoch_c=epoch_c, save=losses_dir, inset=False
)

# Plot results
results_plot_dir = os.path.join(run_dir + "_results")
plot_results_two_layers(
    results_filename, bernoulli=False, epoch_c=epoch_c, save=results_plot_dir
)

# Plot sigmoid
sig_dir = os.path.join(run_dir + "_sigmoid")
plot_sigmoid_outputs(
    train_sig_input=train_sig_inputs,
    val_sig_input=val_sig_inputs,
    train_sig_output=train_sig_outputs,
    val_sig_output=val_sig_outputs,
    epoch_c=epoch_c,
    save=sig_dir,
)


# Plot training_accs
training_accs_dir = os.path.join(run_dir + "_training_accs")
plot_training_accs(training_accs, epoch_c=epoch_c, save=training_accs_dir)

Creating new orientation map
[[172.69487   178.23965   133.35286   103.09104   153.8738    141.5435
  176.7622    124.5456    105.62035  ]
 [ 35.97815    71.58959    91.989395   69.425575   71.99754   137.70273
  165.12189   117.94913    25.136189 ]
 [ 67.84493   162.46297   142.57837     9.007031    5.950877   60.928127
   90.58961   162.18724    19.068493 ]
 [ 27.085865  172.8008    160.01271   165.47255    27.742746   50.21197
  143.7793    123.68952   149.20287  ]
 [ 54.598164  139.32262   121.20855   130.1541    143.99326   135.25891
  149.17587   157.32765   174.68413  ]
 [  4.6704493  19.878267  113.98101   134.3164     74.109085    8.544313
  157.88673    76.33787   120.63373  ]
 [176.80812    29.161428   87.669754   79.10616    64.27215   155.54132
  168.56268    41.841663   55.3378   ]
 [ 16.996605  162.2735    167.24075   160.00632    21.834236  138.0646
  142.49863    14.115802  175.33542  ]
 [ 63.618965  155.439     162.59659    16.884869    5.497428   54.782425
  111.3182

TypeError: mul got incompatible shapes for broadcasting: (250,), (25,).